In [1]:
import sqlite3
import pandas as pd
import re
import numpy as np

選手成績に記載されているチーム名は勝敗表のチーム名の略称になっている。変換のための辞書を作成する必要がある。

In [2]:
# データベースに接続
conn = sqlite3.connect('players_info.db')
c = conn.cursor()

In [3]:
# テーブルの確認
c.execute('select * from sqlite_master')
c.fetchall()

[('table',
  'bios',
  'bios',
  2,
  'CREATE TABLE "bios" (\n"index" INTEGER,\n  "name" TEXT,\n  "url" TEXT,\n  "ドラフト" TEXT,\n  "ポジション" TEXT,\n  "投打" TEXT,\n  "生年月日" TEXT,\n  "経歴" TEXT,\n  "身長／体重" TEXT\n)'),
 ('index',
  'ix_bios_index',
  'bios',
  3,
  'CREATE INDEX "ix_bios_index"ON "bios" ("index")'),
 ('table',
  'pitcher_score',
  'pitcher_score',
  293,
  'CREATE TABLE "pitcher_score" (\n"index" INTEGER,\n  "年度" TEXT,\n  "所属球団" TEXT,\n  "登板" REAL,\n  "勝利" REAL,\n  "敗北" REAL,\n  "セーブ" REAL,\n  "H" REAL,\n  "HP" REAL,\n  "完投" REAL,\n  "完封勝" REAL,\n  "無四球" REAL,\n  "勝率" REAL,\n  "打者" REAL,\n  "投球回" TEXT,\n  "安打" REAL,\n  "本塁打" TEXT,\n  "四球" REAL,\n  "死球" REAL,\n  "三振" REAL,\n  "暴投" REAL,\n  "ボーク" REAL,\n  "失点" REAL,\n  "自責点" REAL,\n  "防御率" REAL,\n  "url" TEXT,\n  "name" TEXT\n)'),
 ('index',
  'ix_pitcher_score_index',
  'pitcher_score',
  294,
  'CREATE INDEX "ix_pitcher_score_index"ON "pitcher_score" ("index")'),
 ('table',
  'fielder_score',
  'fielder_score',
  899,
  'CREATE 

In [17]:
# win_loss_tableの構造確認
c.execute('pragma table_info(win_loss_table)')
c.fetchall()

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'URL', 'TEXT', 0, None, 0),
 (2, 'ゲーム差', 'TEXT', 0, None, 0),
 (3, 'チーム', 'TEXT', 0, None, 0),
 (4, 'リーグ', 'TEXT', 0, None, 0),
 (5, '勝利', 'TEXT', 0, None, 0),
 (6, '勝率', 'TEXT', 0, None, 0),
 (7, '年', 'TEXT', 0, None, 0),
 (8, '引分', 'TEXT', 0, None, 0),
 (9, '敗北', 'TEXT', 0, None, 0),
 (10, '試合', 'TEXT', 0, None, 0),
 (11, '順位', 'INTEGER', 0, None, 0),
 (12, '順位前 後期 期', 'TEXT', 0, None, 0)]

In [56]:
# pitcher_scoreの構造確認
c.execute('pragma table_info(pitcher_score)')
c.fetchall()

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, '年度', 'TEXT', 0, None, 0),
 (2, '所属球団', 'TEXT', 0, None, 0),
 (3, '登板', 'REAL', 0, None, 0),
 (4, '勝利', 'REAL', 0, None, 0),
 (5, '敗北', 'REAL', 0, None, 0),
 (6, 'セーブ', 'REAL', 0, None, 0),
 (7, 'H', 'REAL', 0, None, 0),
 (8, 'HP', 'REAL', 0, None, 0),
 (9, '完投', 'REAL', 0, None, 0),
 (10, '完封勝', 'REAL', 0, None, 0),
 (11, '無四球', 'REAL', 0, None, 0),
 (12, '勝率', 'REAL', 0, None, 0),
 (13, '打者', 'REAL', 0, None, 0),
 (14, '投球回', 'TEXT', 0, None, 0),
 (15, '安打', 'REAL', 0, None, 0),
 (16, '本塁打', 'TEXT', 0, None, 0),
 (17, '四球', 'REAL', 0, None, 0),
 (18, '死球', 'REAL', 0, None, 0),
 (19, '三振', 'REAL', 0, None, 0),
 (20, '暴投', 'REAL', 0, None, 0),
 (21, 'ボーク', 'REAL', 0, None, 0),
 (22, '失点', 'REAL', 0, None, 0),
 (23, '自責点', 'REAL', 0, None, 0),
 (24, '防御率', 'REAL', 0, None, 0),
 (25, 'url', 'TEXT', 0, None, 0),
 (26, 'name', 'TEXT', 0, None, 0)]

In [31]:
# 略称と正式名称の対応表を作る。
# 略称の年ごとの一覧の取得
df_team_by_pitcher = pd.read_sql('select distinct 所属球団,年度 from pitcher_score where 年度>=1950', conn)

# 正式名称の一覧の取得
df_team_by_win_loss = pd.read_sql('select distinct チーム,年 from win_loss_table', conn)

# 年情報がobj、strなのでそれを修正する。
df_team_by_pitcher.loc[:, '年度_修正'] = df_team_by_pitcher['年度'].astype(float).astype(int)
df_team_by_win_loss.loc[:, '年_修正'] = df_team_by_win_loss['年'].astype(int)

# 略称はスペースが入っていることがあるので修正する。
df_team_by_pitcher['所属球団_修正'] = df_team_by_pitcher['所属球団'].astype(str).apply(lambda x: re.sub(r'\s', '',str(x)))

# データベースに入っている値がfloatの場合とintの場合があり重複を生んでいる。
# 重複を削除する。
df_team_by_pitcher.drop_duplicates(subset=['年度_修正', '所属球団_修正'], inplace=True)

# 件数の確認。選手情報は2018年の情報がある。
# 勝敗表から取得した件数より12件多い。
player_team_count = df_team_by_pitcher['年度_修正'].value_counts().sum()
print('選手情報から取得した場合、チーム数は{}件です。'.format(player_team_count))
win_loss_table_team_count = df_team_by_win_loss['年_修正'].value_counts().sum()
print('勝敗表から取得した場合、チーム数は{}件です。'.format(win_loss_table_team_count))

選手情報から取得した場合、チーム数は843件です。
勝敗表から取得した場合、チーム数は831件です。


In [30]:
# 年ごとの変換表を作成する。
df_subs = []
years = np.unique(df_team_by_win_loss['年_修正'])
for year in years:
    pitcher_team = df_team_by_pitcher[df_team_by_pitcher['年度_修正']==year]
    win_loss_team = df_team_by_win_loss[df_team_by_win_loss['年_修正']==year]
    name_list = pitcher_team['所属球団_修正'].tolist()
    for name in name_list:
        df_sub = win_loss_team[win_loss_team['チーム'].str.contains(name)]
        # 略称の相手が見つかった
        if df_sub.shape[0]>0:
            df_sub.loc[:, '略称'] = name
        # 阪神は古い年度では大阪タイガースだった
        elif name=='阪神':
            df_sub = win_loss_team[win_loss_team['チーム'].str.contains('タイガース')]
            df_sub.loc[:, '略称'] = name
        # 毎日大映オリオンズの略称は大毎
        elif name=='大毎':
            df_sub = win_loss_team[win_loss_team['チーム'].str.contains('オリオンズ')]
            df_sub.loc[:, '略称'] = name
        
        df_subs.append(df_sub)

# 対応表の作成
correspondence_table = pd.concat(df_subs, axis=0)

/home/kyohei/anaconda2/envs/py36con/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/kyohei/anaconda2/envs/py36con/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [36]:
# データベースに保存するために形を整える。
correspondence_table.drop('年', axis=1, inplace=True)
correspondence_table.rename(columns={'年_修正':'年'}, inplace=True)

ValueError: labels ['年'] not contained in axis

In [38]:
# データベースに保存する。
correspondence_table.to_sql('対応表', conn, if_exists='replace')